<a href="https://colab.research.google.com/github/EthanBerlant/Jedha_Final_Project/blob/main/NLP_Musk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from dateutil.parser import parse
from datetime import datetime

import keras 

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten

from keras.losses import MeanAbsolutePercentageError

from keras.utils import np_utils

from keras.layers.recurrent import LSTM, GRU
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ModelCheckpoint

!git clone 'https://github.com/EthanBerlant/Jedha_Final_Project.git'

fatal: destination path 'Jedha_Final_Project' already exists and is not an empty directory.


In [4]:
output_dim = 128

In [5]:
embed = hub.load("https://tfhub.dev/google/tf2-preview/nnlm-en-dim128-with-normalization/1")

In [6]:
def EMBED(output_dim): # "Token based text embedding trained on English Google News 200B corpus."
  return hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim128-with-normalization/1", output_shape=[output_dim], input_shape=[], dtype=tf.string)

In [7]:
model_Musk_1 = Sequential()

model_Musk_1.add(EMBED(output_dim))
model_Musk_1.add(keras.layers.Dense(128, activation='relu'))
model_Musk_1.add(keras.layers.Dense(64, activation='relu'))
model_Musk_1.add(keras.layers.Dense(32, activation='relu'))
model_Musk_1.add(keras.layers.Dense(1, activation='linear'))

model_Musk_1.compile(loss='MeanAbsolutePercentageError', 
                    optimizer='adam', 
                    metrics=['MeanAbsoluteError'])

#model_Musk_1.summary()

In [8]:
model_Musk_7 = Sequential()

model_Musk_7.add(EMBED(output_dim))
model_Musk_7.add(keras.layers.Dense(128, activation='relu'))
model_Musk_7.add(keras.layers.Dense(64, activation='relu'))
model_Musk_7.add(keras.layers.Dense(32, activation='relu'))
model_Musk_7.add(keras.layers.Dense(1, activation='linear'))

model_Musk_7.compile(loss='MeanAbsolutePercentageError', 
                    optimizer='adam', 
                    metrics=['MeanAbsoluteError'])

#model_Musk_7.summary()

In [9]:
model_Musk_15 = Sequential()

model_Musk_15.add(EMBED(output_dim))
model_Musk_15.add(keras.layers.Dense(128, activation='relu'))
model_Musk_15.add(keras.layers.Dense(64, activation='relu'))
model_Musk_15.add(keras.layers.Dense(32, activation='relu'))
model_Musk_15.add(keras.layers.Dense(1, activation='linear'))

model_Musk_15.compile(loss='MeanAbsolutePercentageError', 
                    optimizer='adam', 
                    metrics=['MeanAbsoluteError'])

#model_Musk_15.summary()

In [10]:
import fnmatch
def filter(tweet):
  new_tweet = []
  for word in tweet.split():
    if word not in fnmatch.filter(tweet.split(), 'http*') and '@' not in word:
      new_tweet.append(word)
  return ' '.join(new_tweet)

In [11]:
tweets = pd.read_json('Jedha_Final_Project/elonmusk.json')
tweets = tweets.drop(['Tweet Id', 'Username'], axis=1).copy()
tweets.columns = ['date', 'text']
tweets['date'] = tweets['date'].apply(datetime.date)
tweets['text'] = tweets['text'].apply(filter)
#tweets['text_embedding'] = tweets['text'].apply(lambda x: embed([x]))

In [12]:
target = pd.read_feather('Jedha_Final_Project/Data/cleaned_dataset.ftr')
SPX = [col for col in target.columns if 'SPX' in col]
SPX.insert(0,'date')
target = target[SPX].copy()
target['date'] = target['date'].apply(datetime.date)
target = target[target['date'] >= tweets['date'].min()]

In [13]:
target_1  = target.copy()
target_7  = target.copy()
target_15 = target.copy()

target_1['date']  = target_1['date'].shift(-1)
target_7['date']  = target_7['date'].shift(-2*7)
target_15['date'] = target_15['date'].shift(-2*15)

target_1  = target_1[['date', 'SPX_close']].dropna()
target_7  = target_7[['date', 'SPX_close-7']].dropna()
target_15 = target_15[['date', 'SPX_close-15']].dropna()

SPX_1  = tweets[['date','text']].merge(target_1, on='date', how='inner')
SPX_7  = tweets[['date','text']].merge(target_7, on='date', how='inner')
SPX_15 = tweets[['date','text']].merge(target_15, on='date', how='inner')

model_Musk_1.layers[-1].bias.assign([SPX_1['SPX_close'].mean()])
model_Musk_7.layers[-1].bias.assign([SPX_7['SPX_close-7'].mean()])
model_Musk_15.layers[-1].bias.assign([SPX_15['SPX_close-15'].mean()])

<tf.Variable 'UnreadVariable' shape=(1,) dtype=float32, numpy=array([3175.9546], dtype=float32)>

In [14]:
callbacks = [tf.keras.callbacks.EarlyStopping(patience=10),
             #tf.keras.callbacks.ModelCheckpoint(filepath='Jedha_Final_Project/Model Weights/model.{epoch:02d}-{val_loss:.2f}.h5',
                                                #save_best_only=True,
                                                #save_weights_only=True,)
]

In [15]:
X_train, X_test, y_train, y_test = train_test_split(SPX_1['text'],
                                                    SPX_1['SPX_close'],
                                                    test_size=0.3,
                                                    random_state=42
                                                    )

model_Musk_1.fit(x=X_train, 
                y=y_train,
                validation_data=(X_test,y_test),
                epochs=600,
                callbacks=callbacks)

Epoch 1/600
256/256 [==============================] - 6s 7ms/step - loss: 22.5353 - mean_absolute_error: 684.4692 - val_loss: 21.6333 - val_mean_absolute_error: 685.7678
Epoch 2/600
256/256 [==============================] - 2s 8ms/step - loss: 21.0847 - mean_absolute_error: 674.6410 - val_loss: 21.2673 - val_mean_absolute_error: 683.1230
Epoch 3/600
256/256 [==============================] - 2s 8ms/step - loss: 20.7741 - mean_absolute_error: 670.7700 - val_loss: 21.0763 - val_mean_absolute_error: 675.5289
Epoch 4/600
256/256 [==============================] - 2s 8ms/step - loss: 20.5304 - mean_absolute_error: 663.5529 - val_loss: 20.9246 - val_mean_absolute_error: 674.6367
Epoch 5/600
256/256 [==============================] - 2s 8ms/step - loss: 20.2973 - mean_absolute_error: 657.0017 - val_loss: 20.8382 - val_mean_absolute_error: 662.5858
Epoch 6/600
256/256 [==============================] - 2s 8ms/step - loss: 20.1243 - mean_absolute_error: 650.6960 - val_loss: 20.8122 - val_mean

In [16]:
X_train, X_test, y_train, y_test = train_test_split(SPX_7['text'],
                                                    SPX_7['SPX_close-7'],
                                                    test_size=0.3,
                                                    random_state=42
                                                    )

model_Musk_7.fit(x=X_train, 
                y=y_train,
                validation_data=(X_test,y_test),
                epochs=600,
                callbacks=callbacks)

Epoch 1/600
256/256 [==============================] - 2s 5ms/step - loss: 22.7345 - mean_absolute_error: 691.7103 - val_loss: 21.9040 - val_mean_absolute_error: 683.4793
Epoch 2/600
256/256 [==============================] - 1s 5ms/step - loss: 21.1611 - mean_absolute_error: 676.5778 - val_loss: 21.4784 - val_mean_absolute_error: 685.0487
Epoch 3/600
256/256 [==============================] - 1s 5ms/step - loss: 20.8697 - mean_absolute_error: 672.3633 - val_loss: 21.3027 - val_mean_absolute_error: 686.2726
Epoch 4/600
256/256 [==============================] - 1s 5ms/step - loss: 20.6426 - mean_absolute_error: 669.6207 - val_loss: 21.2291 - val_mean_absolute_error: 672.2375
Epoch 5/600
256/256 [==============================] - 1s 5ms/step - loss: 20.4743 - mean_absolute_error: 664.5760 - val_loss: 21.0322 - val_mean_absolute_error: 680.3410
Epoch 6/600
256/256 [==============================] - 1s 5ms/step - loss: 20.3331 - mean_absolute_error: 659.9232 - val_loss: 20.9378 - val_mean

In [17]:
X_train, X_test, y_train, y_test = train_test_split(SPX_15['text'],
                                                    SPX_15['SPX_close-15'],
                                                    test_size=0.3,
                                                    random_state=42
                                                    )

model_Musk_15.fit(x=X_train, 
                y=y_train,
                validation_data=(X_test,y_test),
                epochs=600,
                callbacks=callbacks)

Epoch 1/600
256/256 [==============================] - 2s 5ms/step - loss: 22.7076 - mean_absolute_error: 688.1108 - val_loss: 21.8475 - val_mean_absolute_error: 680.8259
Epoch 2/600
256/256 [==============================] - 1s 4ms/step - loss: 21.3002 - mean_absolute_error: 676.4907 - val_loss: 21.4354 - val_mean_absolute_error: 683.9461
Epoch 3/600
256/256 [==============================] - 1s 4ms/step - loss: 20.9171 - mean_absolute_error: 673.3067 - val_loss: 21.3043 - val_mean_absolute_error: 672.1385
Epoch 4/600
256/256 [==============================] - 1s 5ms/step - loss: 20.7026 - mean_absolute_error: 667.7114 - val_loss: 21.0950 - val_mean_absolute_error: 681.3715
Epoch 5/600
256/256 [==============================] - 1s 4ms/step - loss: 20.4932 - mean_absolute_error: 661.6239 - val_loss: 20.9448 - val_mean_absolute_error: 670.5849
Epoch 6/600
256/256 [==============================] - 1s 4ms/step - loss: 20.3147 - mean_absolute_error: 657.2300 - val_loss: 20.8808 - val_mean

In [18]:
min(model_Musk_1.history.history['val_loss'])

20.598237991333008

In [19]:
min(model_Musk_7.history.history['val_loss'])

20.64415740966797

In [20]:
min(model_Musk_15.history.history['val_loss'])

20.645986557006836